<a href="https://colab.research.google.com/github/josepeon/python_dad_class/blob/main/Intro_to_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Networks: Introduction to Neural Nets for Regression and Classification


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
tips = sns.load_dataset('tips')

In [ ]:
tips.head()

In [ ]:
tips.plot.scatter(x = 'total_bill', y = 'tip')

## Building the network with pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
neuron = nn.Linear(in_features=1, out_features=1)

In [ ]:
list(neuron.parameters())

In [ ]:
X = torch.tensor(tips[['total_bill']].values, dtype=torch.float32)

In [ ]:
#scale the data
X =( X - X.mean() )/ X.std()

In [ ]:
X.shape

In [ ]:
neuron(X)

In [ ]:
activation = nn.ReLU()

In [ ]:
loss_function = nn.MSELoss()

In [ ]:
y = torch.tensor(tips[['tip']].values, dtype=torch.float32)

In [ ]:
model = nn.Sequential(neuron, activation)

In [ ]:
list(model.parameters())

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
#step 1: pass data through the model
yhat = model(X)

In [ ]:
#relu(weight x bill + bias)
yhat

In [ ]:
loss = loss_function(y, yhat)

In [ ]:
loss

In [ ]:
optimizer.zero_grad()

In [ ]:
loss.backward()

In [ ]:
optimizer.step()

In [ ]:
#list to keep track of errors
losses = []
#train the network 1000 times (iterations)
for epoch in range(1000):
  #pass the data through the network
  yhat = model(X)
  #evaluate the error in predictions
  loss = loss_function(y, yhat)
  #clear out gradients and backpropogate loss (wiggle weights)
  optimizer.zero_grad()
  loss.backward()
  #track the error
  losses.append(loss.item())
  #weights and biases get adjusted/updated
  optimizer.step()

In [ ]:
plt.plot(losses)
plt.title('Mean Squared Error through training');

In [ ]:
model(X[:10])

In [ ]:
y[:10]

In [ ]:
plt.scatter(tips['total_bill'], tips['tip'])
plt.plot(tips['total_bill'], yhat.detach().numpy(), '-r')

**PROBLEM**

Use the diabetes data below to build a neural net for regression.

In [ ]:
from sklearn.datasets import load_diabetes

In [ ]:
diabetes = load_diabetes(as_frame = True).frame
diabetes.head()

In [ ]:
X = torch.tensor(diabetes.iloc[:, :-1].values, dtype = torch.float32)
y = torch.tensor(diabetes[['target']].values, dtype = torch.float32)

In [ ]:
X.shape

In [ ]:
model = nn.Sequential(nn.Linear(in_features = 10, out_features=1), nn.ReLU())
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
#list to keep track of errors
losses = []
#train the network 1000 times (iterations)
for epoch in range(1000):
  #pass the data through the network
  yhat = model(X)
  #evaluate the error in predictions
  loss = loss_function(y, yhat)
  #clear out gradients and backpropogate loss (wiggle weights)
  optimizer.zero_grad()
  loss.backward()
  #track the error
  losses.append(loss.item())
  #weights and biases get adjusted/updated
  optimizer.step()

In [ ]:
plt.plot(losses)

## Binary Classification

In this situation, we are predicting a binary category.  In the example below, we examine whether or not a tumor is cancerous (class 0) or benign (class 1).  We use all 30 features to predict the `target` column.  The primary differences here are:

- Activation function at end of network is `nn.Sigmoid`
- Loss function is `nn.BCELoss()`

In [ ]:
from sklearn.datasets import load_breast_cancer

In [ ]:
cancer = load_breast_cancer(as_frame=True).frame

In [ ]:
cancer.head()

In [ ]:
model = nn.Sequential(nn.Linear(in_features=30, out_features=1), nn.Sigmoid())

In [ ]:
X = cancer.drop('target', axis = 1).values
y = cancer[['target']].values

In [ ]:
X = torch.tensor(X, dtype = torch.float32)
y = torch.tensor(y, dtype = torch.float32)

In [ ]:
loss_function = nn.BCELoss() #binary classification loss

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)

In [ ]:
X = (X - X.mean())/X.std()

In [ ]:
losses = []
for epoch in range(1000):
  yhat = model(X)
  loss = loss_function(yhat, y)
  optimizer.zero_grad()
  loss.backward()
  losses.append(loss.item())
  optimizer.step()

In [ ]:
plt.plot(losses);

In [ ]:
model(X)[:10] #output of model is probability something is a 1

In [ ]:
predictions = torch.where(model(X) > .5, 1, 0)

In [ ]:
predictions[:5]

In [ ]:
y[:5]

In [ ]:
(y == predictions).sum()/len(y)

**PROBLEM**

Use the orange vs. grapefruit dataset below to build a neural network for classification.

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
fruit = fetch_openml(data_id=43387).frame

In [ ]:
fruit.head()

In [ ]:
fruit.info()

In [ ]:
import numpy as np
X = torch.tensor(fruit.iloc[:, 1:].values, dtype = torch.float32)
y_binary = np.where(fruit[['name']].values == 'orange', 1, 0)

In [ ]:
y_binary

In [ ]:
y = torch.tensor(y_binary, dtype = torch.float32)

In [ ]:
#model??
model = nn.Sequential(nn.Linear(in_features = 5, out_features=1), nn.Sigmoid())
loss_function = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)

In [ ]:
X = (X - X.mean())/X.std()

In [ ]:
losses = []
for epoch in range(1000):
  yhat = model(X)
  loss = loss_function(yhat, y)
  optimizer.zero_grad()
  loss.backward()
  losses.append(loss.item())
  optimizer.step()

In [ ]:
plt.plot(losses)

In [ ]:
yhat = torch.where(model(X) > .5, 1, 0)

In [ ]:
(yhat == y).sum()/len(y)

In [ ]:
torch.cuda.is_available()

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
device

In [ ]:
X = X.to(device)

## Multi-class classification

In [ ]:
from torchvision.datasets import MNIST

In [ ]:
train = MNIST('.', download = True)

In [ ]:
plt.imshow(train[0][0])

In [ ]:
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import DataLoader

In [ ]:
#turn image into pytorch tensor
#then scale the tensor
transform = Compose([ToTensor(), Normalize(mean = (0.5), std = (0.5))])

In [ ]:
train = MNIST('.', download = True, transform=transform)

In [ ]:
train

In [ ]:
#create smaller batches of data for training
trainloader = DataLoader(train, batch_size=24)

In [ ]:
model = nn.Sequential(nn.Flatten(),
                      nn.Linear(in_features = 28*28, out_features=100),
                      nn.ReLU(),
                      nn.Linear(100, 10))

In [ ]:
loss_function = nn.CrossEntropyLoss()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [ ]:
losses = []
for epoch in range(10):
  for X, y in trainloader:
    yhat = model(X)
    loss = loss_function(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  losses.append(loss.item())

In [ ]:
plt.plot(losses)

In [ ]:
correct = 0
total = 0
for X, y in trainloader:
  yhat = torch.argmax(model(X), dim = 1)
  correct += (y == yhat).sum()
  total += len(y)

In [ ]:
correct/total

In [ ]:
y

### Train it on the GPU

In [ ]:
device = 'cuda'

In [ ]:
#move everything to the GPU
model = model.to(device)
losses = []
for epoch in range(20):
  for X, y in trainloader:
    X =  X.to(device)
    y = y.to(device)
    yhat = model(X)
    loss = loss_function(yhat, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  losses.append(loss.item())

In [ ]:
plt.plot(losses)

In [ ]:
model

In [ ]:
correct = 0
total = 0
for X, y in trainloader:
  X =  X.to(device)
  y = y.to(device)
  yhat = torch.argmax(model(X), dim = 1)
  correct += (y == yhat).sum()
  total += len(y)

In [ ]:
correct/total